In [1]:
from PyPDF2 import PdfReader, PdfWriter

reader = PdfReader("../debu.pdf")
writer = PdfWriter()

for page in reader.pages:
    page.compress_content_streams()  # This is CPU intensive!
    writer.add_page(page)

with open("out.pdf", "wb") as f:
    writer.write(f)

In [7]:
import PyPDF2

# Open the PDF file in binary mode
with open('../debu.pdf', 'rb') as file:
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(file)

    # Create a PDF writer object
    pdf_writer = PyPDF2.PdfWriter()

    # Iterate over all pages in the PDF document
    for page_num in range(len(pdf_reader.pages)):
        # Get the page object
        page_obj = pdf_reader.pages[page_num]

        # Compress the page
        page_obj.compress_content_streams()

        # Add the compressed page to the PDF writer object
        pdf_writer.add_page(page_obj)

    # Open a new file to write the compressed PDF data
    with open('output.pdf', 'wb') as output_file:
        # Write the compressed PDF data to the output file
        pdf_writer.write(output_file)


In [34]:
import io
from PyPDF2 import PdfReader, PdfWriter
import zlib


def compress_pdf_with_pypdf2(input_path):
    with open(input_path, 'rb') as f:
        pdf_reader = PdfReader(f)
        pdf_writer = PdfWriter()

        for i in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[i]
            page_content = io.BytesIO()
            page.compress_content_streams()
            compressed_page = pdf_writer.add_blank_page(
                page.mediabox.width, page.mediabox.height)
            compressed_page.merge_page(PdfReader(page_content).read(0))

        with open('output.pdf', 'wb') as output_file:
            # Write the compressed PDF data to the output file
            pdf_writer.write(output_file)
compress_pdf_with_pypdf2("../debu.pdf")

EmptyFileError: Cannot read an empty file